# Supervised Models

In [ ]:
import pandas as pd

In [ ]:
## import df

In [ ]:
df_reviews_final = pd.read_csv('final_df.csv')

In [ ]:
df_reviews_final.dropna(inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split
X = df_reviews_final[['reviews','asin_reviewer','review_length']]
y = df_reviews_final['overall']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [ ]:
tfidf = TfidfVectorizer(max_features=7500, token_pattern = '(?ui)\\b\\w*[a-z]+\\w*\\b', analyzer='word')
tfidf_words = tfidf.fit_transform(X_train['reviews'])
vector_df = pd.DataFrame(tfidf_words.toarray(), columns=tfidf.get_feature_names())

In [ ]:
## convert test data into vectors as well

In [ ]:
tfidf_test = tfidf.transform(X_test['reviews'])

In [ ]:
X_test

In [ ]:
vector_df_test = pd.DataFrame(tfidf_test.toarray(), columns=tfidf.get_feature_names())

## LSA

In [ ]:
## try bag of words later???

In [ ]:
# vectorize the data and get check what the top words
tf_idf_vectorize = TfidfVectorizer(max_df=0.5, max_features=10000,
                                  min_df=2, stop_words='english',
                                  use_idf=True)

tf_idf = tf_idf_vectorize.fit_transform(cleaned_reviews)

In [ ]:
# recommended components for LSA
SVD = TruncatedSVD(n_components=100, random_state=1234)

SVD_matrix = SVD.fit_transform(tf_idf)

LSA = Normalizer(copy=False).fit_transform(SVD_matrix)

In [ ]:
# 100 rows - these are the features created after applying SVD which reduces the dimensionality of our feature space
# but maintains
LSA_df = pd.DataFrame(SVD.components_, columns=tf_idf_vectorize.get_feature_names())

In [ ]:
for compNum in range(0, 10):

    comp = SVD.components_[compNum]
    
    # Sort the weights in the first component, and get the indeces
    indeces = numpy.argsort(comp).tolist()
    
    # Reverse the indeces, so we have the largest weights first.
    indeces.reverse()
    
    # Grab the top 10 terms which have the highest weight in this component.        
    terms = [features[weightIndex] for weightIndex in indeces[0:10]]    
    weights = [comp[weightIndex] for weightIndex in indeces[0:10]]    
   
    # Display these terms and their weights as a horizontal bar graph.    
    # The horizontal bar graph displays the first item on the bottom; reverse
    # the order of the terms so the biggest one is on top.
    terms.reverse()
    weights.reverse()
    positions = arange(10) + .5    # the bar centers on the y axis
    
    figure(compNum)
    barh(positions, weights, align='center')
    yticks(positions, terms)
    xlabel('Weight')
    title('Strongest terms for component %d' % (compNum))
    grid(True)
    show()

## Modeling

In [ ]:
# dummy classifier to start

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
from sklearn.metrics import confusion_matrix

In [ ]:
dum_clf = DummyClassifier()
dum_model = dum_clf.fit(vector_df,y_train)
y_hat_test = dum_model.predict(vector_df_test)

In [ ]:
print(accuracy_score(y_hat_test,y_test))

In [ ]:
# Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
NB = MultinomialNB()
NB_model = NB.fit(vector_df, y_train)

In [ ]:
y_hat_test_NB = NB.predict(vector_df_test)

In [ ]:
print(accuracy_score(y_hat_test_NB,y_test))

In [ ]:
cnf_matrix = confusion_matrix(y_test, y_hat_test_NB)

plt.figure()
plot_confusion_matrix(cnf_matrix, classes=[1,2,3,4,5],normalize=True,
                      title='Confusion matrix')
plt.show()

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    #Add Normalization Option
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes)
    plt.yticks(tick_marks, classes)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig('confusion')

In [ ]:
## Logistic Reg

In [ ]:
logreg = LogisticRegression(fit_intercept = False, solver='liblinear')
model_log = logreg.fit(vector_df, y_train)
model_log

In [ ]:
y_hat_test_log = model_log.predict(vector_df_test)

In [ ]:
print(accuracy_score(y_hat_test_log,y_test))